<a href="https://colab.research.google.com/github/Karthick47v2/mock-buddy/blob/base-dev/model_train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


### Import libs


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tensorflow import keras

from google.colab import drive
drive.mount('/content/gdrive')

!unzip gdrive/MyDrive/300cw/300w_96.zip


### Initialize data generator


In [ ]:
train_df = pd.read_csv('train.csv')
img_size = 96
no_channel = 1
batch_size = 64

data_gen = keras.preprocessing.image.ImageDataGenerator(validation_split=0.2)
train_data_gen = data_gen.flow_from_dataframe(
    dataframe=train_df,
    directory='./',
    x_col='path',
    y_col=[
        f"{i}" for i in range(136)],
    class_mode='raw',
    subset='training',
    batch_size=batch_size,
    seed=42,
    shuffle=True,
    target_size=(img_size, img_size),
    color_mode='grayscale'
)

validation_data_gen = data_gen.flow_from_dataframe(
    dataframe=train_df,
    directory='./',
    x_col='path',
    y_col=[
        f"{i}" for i in range(136)],
    class_mode='raw',
    subset='validation',
    batch_size=batch_size,
    target_size=(
        img_size, img_size),
    color_mode='grayscale'
)


### Callback functions


In [4]:
model_checkpoint = keras.callbacks.ModelCheckpoint(
    'checkpoint',
    monitor='val_root_mean_squared_error',
    save_best_only=True,
    save_weights_only=False,
    mode='min',
    save_freq="epoch",
)


def scheduler(epoch, lr):
    if epoch < 10:
        return lr
    elif (epoch % 10) == 0:
        return lr * 0.9
    return lr


shd = keras.callbacks.LearningRateScheduler(scheduler)


### Model


In [ ]:
model_input = keras.Input(shape=(img_size, img_size, no_channel))

# # block_1
x = keras.layers.Conv2D(filters=16, kernel_size=(
    3, 3), kernel_initializer='he_uniform', activation='relu', padding='same')(model_input)
x = keras.layers.BatchNormalization()(x)

x = keras.layers.Conv2D(filters=16, kernel_size=(
    3, 3), kernel_initializer='he_uniform', activation='relu', padding='same')(x)
x = keras.layers.BatchNormalization()(x)

x = keras.layers.Conv2D(filters=16, kernel_size=(
    3, 3), kernel_initializer='he_uniform', activation='relu', padding='same')(x)
x = keras.layers.MaxPooling2D(pool_size=(2, 2))(x)


# # block_1
x = keras.layers.Conv2D(filters=32, kernel_size=(
    3, 3), kernel_initializer='he_uniform', activation='relu', padding='same')(x)
x = keras.layers.BatchNormalization()(x)

x = keras.layers.Conv2D(filters=32, kernel_size=(
    3, 3), kernel_initializer='he_uniform', activation='relu', padding='same')(x)
x = keras.layers.BatchNormalization()(x)

x = keras.layers.Conv2D(filters=32, kernel_size=(
    3, 3), kernel_initializer='he_uniform', activation='relu', padding='same')(x)
x = keras.layers.MaxPooling2D(pool_size=(2, 2))(x)
x = keras.layers.BatchNormalization()(x)


# # block_2
x = keras.layers.Conv2D(filters=48, kernel_size=(
    3, 3), kernel_initializer='he_uniform', activation='relu', padding='same')(x)
x = keras.layers.BatchNormalization()(x)

x = keras.layers.Conv2D(filters=48, kernel_size=(
    3, 3), kernel_initializer='he_uniform', activation='relu', padding='same')(x)
x = keras.layers.BatchNormalization()(x)

x = keras.layers.Conv2D(filters=48, kernel_size=(
    3, 3), kernel_initializer='he_uniform', activation='relu', padding='same')(x)
x = keras.layers.MaxPooling2D(pool_size=(2, 2))(x)
x = keras.layers.BatchNormalization()(x)


# # block_3
x = keras.layers.Conv2D(filters=64, kernel_size=(
    3, 3), kernel_initializer='he_uniform', activation='relu', padding='same')(x)
x = keras.layers.BatchNormalization()(x)

x = keras.layers.Conv2D(filters=64, kernel_size=(
    3, 3), kernel_initializer='he_uniform', activation='relu', padding='same')(x)
x = keras.layers.BatchNormalization()(x)

x = keras.layers.Conv2D(filters=64, kernel_size=(
    3, 3), kernel_initializer='he_uniform', activation='relu', padding='same')(x)
x = keras.layers.MaxPooling2D(pool_size=(2, 2))(x)
x = keras.layers.BatchNormalization()(x)


# # block_4
x = keras.layers.Conv2D(filters=80, kernel_size=(
    3, 3), kernel_initializer='he_uniform', activation='relu', padding='same')(x)
x = keras.layers.BatchNormalization()(x)

x = keras.layers.Conv2D(filters=80, kernel_size=(
    3, 3), kernel_initializer='he_uniform', activation='relu', padding='same')(x)
x = keras.layers.BatchNormalization()(x)

x = keras.layers.Conv2D(filters=80, kernel_size=(
    3, 3), kernel_initializer='he_uniform', activation='relu', padding='same')(x)
x = keras.layers.MaxPooling2D(pool_size=(2, 2))(x)
x = keras.layers.BatchNormalization()(x)


# # last block
x = keras.layers.GlobalAveragePooling2D()(x)
x = keras.layers.Dense(units=136, kernel_initializer='he_uniform')(x)

model = keras.Model(model_input, x)

model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.004),
              loss='mean_squared_error', metrics=[keras.metrics.RootMeanSquaredError()])

model.summary()


### Train model


In [ ]:
history = model.fit(train_data_gen, epochs=200, batch_size=batch_size,
                    validation_data=validation_data_gen, callbacks=[shd, model_checkpoint])


### Test


In [54]:
import cv2

my_model = keras.models.load_model('checkpoint')

# challenging dataset
test_df = pd.read_csv('test_ibug.csv')

# # common dataset
# helen_test_df, lfpw_test_df = pd.read_csv('test_helen.csv'), pd.read_csv('test_lfpw.csv')
# test_df = pd.concat([helen_test_df, lfpw_test_df], axis=0)

image = [cv2.imread('./' + row.iloc[0], flags=cv2.IMREAD_GRAYSCALE).reshape(96, 96, 1)
         for idx, row in test_df.iterrows()]

y_hat = []
for img in image:
    y_hat.append(my_model.predict(np.expand_dims(img, 0)))


val = 0
result = []
i = 0
for idx, row in test_df.iterrows():
    y_true, y_pred = row.iloc[1:137].values.reshape(
        68, 2), y_hat[i].reshape(68, 2)
    i += 1
    iod = np.linalg.norm(y_true[36]-y_true[45])
    m = 0
    for j in range(68):
        m = m + np.linalg.norm(y_pred[j]-y_true[j])
    result.append(m / (68 * iod))


In [ ]:
print(f"Cumalative: {sum(result)}")
print(f"Avg: {sum(result)/len(y_hat)}")


### Evaluate


In [ ]:
import cv2
my_model = keras.models.load_model('checkpoint')

train_df = pd.read_csv('test.csv')
image = cv2.imread(
    './' + train_df.iloc[100, 0], flags=cv2.IMREAD_GRAYSCALE).reshape(96, 96, 1)
plt.imshow(image.reshape(96, 96))
y_hat = (my_model.predict(np.expand_dims(image, 0)))
plt.scatter(y_hat[0][:136:2], y_hat[0][1:136:2], s=3, c='r')
